In [1]:
!nvidia-smi

Thu Jul 10 15:37:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 34%   44C    P8             N/A /  115W |    1221MiB /   8188MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 📌 라이브러리

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import gc

In [3]:
# 📌 메모리 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='float64').columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include='int64').columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

In [4]:
# 📌 1. 테스트 데이터 불러오기
test = pd.read_csv('병합/merged_final_test_셀_추가병합.csv')

In [5]:
# ID 저장 후 불필요 컬럼 제거
test_id = test['ID']

if 'Unnamed: 0' in test.columns:
    test = test.drop(columns=['Unnamed: 0'])

if 'ID' in test.columns:
    test = test.drop(columns=['ID'])

In [6]:
# 📌 2. 메모리 최적화 및 타입 변환
test = optimize_types(test)

# object → category 변환
for col in test.select_dtypes(include='object').columns:
    test[col] = test[col].astype('category')

In [7]:
with open('XGBoost/X_columns.pkl', 'rb') as f:
    X_columns = pickle.load(f)

In [8]:
test = test[X_columns]

In [9]:
with open('XGBoost/final_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
preds_proba = model.predict_proba(test)
preds_class = np.argmax(preds_proba, axis=1)

In [11]:
# 숫자 라벨을 문자 라벨로 매핑
label_map = np.array(['A', 'B', 'C', 'D', 'E'])  # le.classes_ 순서

preds_labels = label_map[preds_class]

In [12]:
submission = pd.DataFrame({
    'ID': test_id,
    'Segment': preds_labels
})

submission.to_csv('XGBoost/submission.csv', index=False, encoding='utf-8-sig')

print("✅ 제출 파일 저장 완료")

✅ 제출 파일 저장 완료


In [13]:
# 열어서 확인
submission = pd.read_csv('XGBoost/submission.csv')
submission

,ID,Segment
0,TEST_00000,D
1,TEST_00001,E
2,TEST_00002,D
3,TEST_00003,E
4,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,C


In [14]:
print("컬럼 목록:", submission.columns.tolist())
print("총 샘플 수:", submission.shape[0])

컬럼 목록: ['ID', 'Segment']
총 샘플 수: 600000


In [15]:
# 마지막 100,000개만 추출
submission_tail = submission.tail(100000)

# 제출 파일 저장
submission_tail.to_csv(
    'XGBoost/submission_tail.csv',
    index=False,
    encoding='utf-8-sig'
)

print("✅ 마지막 100,000개만 제출 파일 저장 완료")

✅ 마지막 100,000개만 제출 파일 저장 완료


In [16]:
print(submission_tail['Segment'].value_counts())
print(submission_tail.shape)

Segment
E    73677
D    18842
C     7445
A       34
B        2
Name: count, dtype: int64
(100000, 2)
